In [ ]:
# Import packages
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import pandas as pd

In [ ]:
data_type = 'Data245'
project_dir = 'C:\\Users\\Na Min An\\Desktop\\Development\\Fluorescence'
calculate = True
visualize = False

if data_type in ['Data099', 'Data101', 'Data117', 'Data118', 'Data211', 'Data213', 'Data217', 'Data218']:
    is_vertical = True
elif data_type in ['Data239', 'Data240', 'Data244', 'Data245']:
    is_vertical = False

data_dir = os.path.join(os.path.join(project_dir, 'data'), 'AxonLengthArea')
fig_dir = os.path.join(os.path.join(project_dir, 'figures'), 'AxonLengthArea')
if visualize:
    os.makedirs(os.path.join(fig_dir, data_type), exist_ok=True)
background_path = os.path.join(data_dir, f'{data_type}.tif') 
image = Image.open(background_path)
width, height = image.size
left = 100
top = 0
right = width - left
bottom = height - 20
image = image.crop((left, top, right, bottom)).resize((322, 322))

backarr = np.array(image)
plt.imshow(backarr)
plt.show()
assert backarr.shape[0] == backarr.shape[1]

axon_colors = [(255, 38, 0), # Axon 0
                (0, 225, 255), # Axon 1
                (229, 255, 0), # Axon 2
                (0, 38, 255)] # Axon 3

time_folder = os.path.join(data_dir, data_type)
full_len_dic = {'Axon':[], 'Length':[]}
len_dic = {'Axon':[], 'Time':[], 'Length':[]}
area_dic = {'Time':[], 'Area':[]}

for a in tqdm(range(len(axon_colors))):
    axonarr = np.zeros((backarr.shape[0], backarr.shape[1])) 
    for i in range(backarr.shape[0]):
        for j in range(backarr.shape[1]):
            parirr = backarr[i, j]
            if tuple(parirr) == axon_colors[a]: 
                axonarr[i,j] = 255. 
    axons = cv.findNonZero(axonarr*np.ones(axonarr.shape)).squeeze()

    df = pd.DataFrame(axons, columns=['x', 'y'])   
    if is_vertical:
        axis1, axis2 = 'x', 'y'
    else:
        axis1, axis2 = 'y', 'x'
 
    df = df.groupby(axis2).agg('mean').reset_index()
    axon_l = 0
    for idx in range(df.shape[0]-1):
        x1, y1 = df['x'][idx], df['y'][idx]
        x2, y2 = df['x'][idx+1], df['y'][idx+1]
        x = x2 - x1
        y = y2 - y1
        l = np.linalg.norm((x, y))
        axon_l += l
    axon_l *= (6.6 / (backarr.shape[0] / 80)) # 1 pixel = 6.6 um in 80x80 
    full_len_dic['Axon'].append(a+1)
    full_len_dic['Length'].append(axon_l)

    if calculate:
        for time_file in tqdm(os.listdir(time_folder), leave=False):
            frame_num = time_file.split('.')[0]
            image = Image.open(os.path.join(time_folder, time_file)).convert("L")
            width, height = image.size
            left = 5
            top = 30
            right = width - left
            bottom = (right - left) + top
            image = image.crop((left, top, right, bottom))
            imarr = np.array(image)
            binary = np.where(imarr > 0, 255, imarr)
            binary = np.uint8(binary)
            canny = cv.Canny(binary, 100, 200).astype(float)

            try:
                axons = cv.findNonZero(axonarr*binary).squeeze()
                if visualize:
                    plt.imshow(axonarr*binary, cmap='gray')
                    plt.savefig(os.path.join(os.path.join(fig_dir, data_type), f'time{frame_num}_axon{a+1}.png'))
                    plt.show()

                df = pd.DataFrame(axons, columns=['x', 'y'])
                if is_vertical:
                    axis1, axis2 = 'x', 'y'
                else:
                    axis1, axis2 = 'y', 'x'

                df = df.groupby(axis2).agg('mean').reset_index()
                sum_l = 0
                for idx in range(df.shape[0]-1):
                    x1, y1 = df['x'][idx], df['y'][idx]
                    x2, y2 = df['x'][idx+1], df['y'][idx+1]
                    x = x2 - x1
                    y = y2 - y1
                    l = np.linalg.norm((x, y))
                    sum_l += l
                sum_l *= (6.6 / (backarr.shape[0] / 80)) # 1 pixel = 6.6 um in 80x80
            except:
                print(f'No activations spread for {frame_num}')
                sum_l = 0

            len_dic['Axon'].append(a+1)
            len_dic['Time'].append(frame_num)
            len_dic['Length'].append(sum_l)

            if a == 0:
                try:
                    sum_a = cv.findNonZero(binary).shape[0] * (6.6 ** 2) # 1 pixel = (6.6 * 6.6) um 
                except:
                    sum_a = 0
                area_dic['Time'].append(frame_num)
                area_dic['Area'].append(sum_a)
                
full_len_df = pd.DataFrame.from_dict(full_len_dic)
full_len_df.to_csv(os.path.join(fig_dir, f'Full_len_{data_type}.csv'), index=False)

if calculate:
    len_df = pd.DataFrame.from_dict(len_dic)
    len_df = len_df.pivot(index='Time', columns='Axon', values='Length')
    len_df.to_csv(os.path.join(fig_dir, f'Len_{data_type}.csv'), index=True)
    area_df = pd.DataFrame.from_dict(area_dic)
    area_df.to_csv(os.path.join(fig_dir, f'Area_{data_type}.csv'), index=False)